In [3]:
# import
import numpy as np
import tensor.tensor_product_wrapper as tp
from utils.plotting_utils import montage_array, slice_subplots
import matplotlib.pyplot as plt
import similarity_metrics as sm
from sklearn.model_selection import train_test_split
import scipy.io
import utils.starplus_utils as starp
from numpy.linalg import norm
from tensor.utils import assert_compatile_sizes_modek, reshape, make_axis_iterable
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
from matplotlib.cm import ScalarMappable
from numpy.linalg import svd
from tensor.mode_k import modek_unfold
from sklearn.utils.extmath import randomized_svd
from scipy.stats import ortho_group

In [4]:
# ==================================================================================================================== #
# choose product type {'f', 't', 'c'，'m'}
# m-product using haarMatrix
prod_type = 'm'


#choose M {'band','haar','data dependent','random orthogonal'}
#ONLY USING M-PRODUCT

In [5]:
# ==================================================================================================================== #
# define projection
def projection_m_prod_type(A, U, M):
    if M == 'band':
        # Banded Matrix
        training_coeff = tp.ten_prod(tp.ten_tran(U, prod_type=prod_type), A, prod_type=prod_type,M = (New_M_Matrix(64,8),New_M_Matrix(8,2),New_M_Matrix(16,3)))
        return tp.ten_prod(U, training_coeff, prod_type=prod_type, M = (New_M_Matrix(64,8),New_M_Matrix(8,2),New_M_Matrix(16,3)))
    if M == 'haar':
        # Haar Matrix
        training_coeff = tp.ten_prod(tp.ten_tran(U, prod_type=prod_type), A, prod_type=prod_type, M = (haar_normalized(64),haar_normalized(8),haar_normalized(16)))
        return tp.ten_prod(U, training_coeff, prod_type=prod_type, M = (haar_normalized(64),haar_normalized(8),haar_normalized(16)))
    if M == 'data dependent':
        # Data Dependent Matrix
        u1, _,_ = randomized_svd(modek_unfold(training_data[:, :, :, :, :],2), n_components=64, n_iter=5, random_state=20)
        u2,_,_ = randomized_svd(modek_unfold(training_data[:, :, :, :, :],3), n_components=8, n_iter=5, random_state=20)
        u3,_,_ = randomized_svd(modek_unfold(training_data[:, :, :, :, :],4), n_components=16, n_iter=5, random_state=20)
        training_coeff = tp.ten_prod(tp.ten_tran(U, prod_type=prod_type), A, prod_type=prod_type, M = (u1,u2,u3))
        return tp.ten_prod(U, training_coeff, prod_type=prod_type, M = (u1,u2,u3))
    if M == 'random orthogonal':
        # Random Orthogonal Matrix
        training_coeff = tp.ten_prod(tp.ten_tran(U, prod_type=prod_type), A, prod_type=prod_type,M = (ortho_group.rvs(64),ortho_group.rvs(8),ortho_group.rvs(16)))
        return tp.ten_prod(U, training_coeff, prod_type=prod_type, M = (ortho_group.rvs(64),ortho_group.rvs(8),ortho_group.rvs(16)))

In [6]:
# ==================================================================================================================== #
# for reproducibility
np.random.seed(20)

In [7]:
# load data
# we need the variables
#   training_data, training_labels, test_data, test_labels, num_classes
num_classes = 2
star_plus_data = scipy.io.loadmat('data-starplus-04847-v7.mat')
tensor_PS, labels = starp.get_labels(star_plus_data)
tensor_PS  = tensor_PS / norm(tensor_PS)

In [8]:
print(np.transpose(labels).shape)
print(np.moveaxis(tensor_PS, -1, 0).shape)
training_data, test_data, training_labels, test_labels = train_test_split(np.moveaxis(tensor_PS, -1, 0), np.transpose(labels), test_size=0.33, random_state=42)
(unique, counts) = np.unique(test_labels, return_counts=True)
(unique, counts) = np.unique(training_labels, return_counts=True)

(80, 1)
(80, 64, 64, 8, 16)


In [9]:
# move the label number to second axis
training_data = np.moveaxis(training_data, 0, 1)
test_data = np.moveaxis(test_data, 0, 1)
print(training_data.shape)
print(test_data.shape)

# create the boolean array for training and testing
boolean_list = []
for i in (training_labels):
    boolean_list.append(i[0])
boolean_array_training = np.asarray(boolean_list)
print(boolean_array_training)

boolean_list = []
for i in (test_labels):
    boolean_list.append(i[0])
boolean_array_testing = np.asarray(boolean_list)
print(boolean_array_testing)

(64, 53, 64, 8, 16)
(64, 27, 64, 8, 16)
[1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1.
 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 1. 1. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 1.]


In [10]:
# HaarMatrix utilized for m-product
def haarMatrix(n):
    # n is the power of 2
    if n > 2:
        M = haarMatrix(n / 2)
    else:
        return np.array([[1, 1], [1, -1]])
    M_n = np.kron(M, [1, 1])
    M_i = np.sqrt(n/2)*np.kron(np.eye(len(M)), [1, -1])
    M = np.vstack((M_n, M_i))
    return M
def haar_normalized(n):
    M = haarMatrix(n)
    M = M/np.sqrt(np.sum(M[0]))
    return M

def New_M_Matrix(dimension,bandwidth):
    A = np.eye(dimension)
    for i in range(0, -bandwidth, -1):
        A += np.eye(dimension, k = i -1)
    for i in range(dimension):
        temp = np.sum(A[i])
        for k in range(dimension):
            if A[i][k] == 1:
                A[i][k] = 1/temp
    return A



In [11]:
# ==================================================================================================================== #
# form local t-svd
# num_class should be 2
#num_classes = len(np.unique(training_labels))
#k = 5

#U = []
def create_basis(M_choice,num_classes,k,training_data,prod_type):
    U = []
    #print(num_classes)
    for i in range(0,num_classes):
        if M_choice == 'band':
            # Bandwidth Matrix
            u, _, _, _ = tp.ten_svd(training_data[:, boolean_array_training == i, :], k, prod_type=prod_type, M = (New_M_Matrix(64,8),New_M_Matrix(8,2),New_M_Matrix(16,3)))
            U.append(u)
        elif M_choice == 'haar':
            # Haar Matrix
            u, _, _, _ = tp.ten_svd(training_data[:, boolean_array_training == i, :], k, prod_type=prod_type, M = (haar_normalized(64),haar_normalized(8),haar_normalized(16)))
            U.append(u)
        elif M_choice == 'data dependent':
            # Data Dependent Matrix
            u1, _,_ = randomized_svd(modek_unfold(training_data[:, :, :, :, :],2), n_components=64, n_iter=5, random_state=20)
            u2,_,_ = randomized_svd(modek_unfold(training_data[:, :, :, :, :],3), n_components=8, n_iter=5, random_state=20)
            u3,_,_ = randomized_svd(modek_unfold(training_data[:, :, :, :, :],4), n_components=16, n_iter=5, random_state=20)
            u, _, _, _ = tp.ten_svd(training_data[:, boolean_array_training == i, :], k, prod_type=prod_type, M = (u1,u2,u3))
            U.append(u)
        elif M_choice == 'random orthogonal':
            # Random Orthogonal Matrix
            u, _, _, _ = tp.ten_svd(training_data[:, boolean_array_training == i, :], k, prod_type=prod_type, M = (ortho_group.rvs(64),ortho_group.rvs(8),ortho_group.rvs(16)))
            U.append(u)
    
    return U
    
def create_test_error(M_choice,U,k,num_classes,boolean_array_testing,test_data,prod_type='m'):
    #initialize test_error
    test_error = np.zeros([num_classes, test_data.shape[1]])
    
    #find error
    for i in range(num_classes):
        test_projection = projection_m_prod_type(test_data, U[i], M_choice)
        test_error[i, :] = sm.frobenius_metric(test_data, test_projection, axis=1)
        
    #classification
    test_predicted_classes = np.argmin(test_error, axis=0).reshape(-1)
    
    test_num_correct = np.sum(test_predicted_classes == boolean_array_testing)
    test_error = test_num_correct / test_data.shape[1]
    print('M = {}, k = {}'.format(M_choice, k))
    print('test accuracy = %0.2f' % (100 * test_error))
    return test_error

In [ ]:
def find_best_accuracy_for_given_M(M_choice,num_classes,training_data,test_data,k_list):
    prod_type = 'm'
    errors_for_given_M_list = []
    for i in range(0,len(k_list)):
        k = k_list[i]
        U = create_basis(M_choice,num_classes,k,training_data,prod_type)
        test_error = create_test_error(M_choice,U,k,num_classes,boolean_array_testing,test_data,prod_type)
        errors_for_given_M_list.append(test_error)
    return errors_for_given_M_list

In [ ]:
import pandas as pd
M_choices = ['haar','band','data dependent','random orthogonal']
k_list = np.arange(1,11,1)

column_names = ['M','k','error']
test_error_df = pd.DataFrame(columns=column_names)
#print(test_error_df)
for M_choice in M_choices:
    print(M)
    num_classes = len(np.unique(training_labels))
    M_list = [M_choice] * k_list.shape[0]
    #print(M_list)
    error_for_given_M_list = find_best_accuracy_for_given_M(M_choice,num_classes,training_data,test_data,k_list)
    print(error_for_given_M_list)
    #creating dataframe rows
    M_errors = pd.DataFrame(
    {'M' : M_list,
    'k' : k_list,
    'error': error_for_given_M_list}
    )
    test_error_df = test_error_df.append(M_errors)
    
print(test_error_df)

In [ ]:
grouped_df = test_error_df.groupby('M')
maximums = grouped_df['error'].max()
maximums = maximums.reset_index()
print(maximums)

In [15]:
print(test_error_df)

                   M   k     error
0               haar   1  0.851852
1               haar   2  0.888889
2               haar   3  0.888889
3               haar   4  0.925926
4               haar   5  0.888889
5               haar   6  0.888889
6               haar   7  0.888889
7               haar   8  0.851852
8               haar   9  0.814815
9               haar  10  0.851852
0               band   1  0.592593
1               band   2  0.518519
2               band   3  0.407407
3               band   4  0.555556
4               band   5  0.555556
5               band   6  0.555556
6               band   7  0.555556
7               band   8  0.407407
8               band   9  0.518519
9               band  10  0.555556
0     data dependent   1  0.814815
1     data dependent   2  0.666667
2     data dependent   3  0.666667
3     data dependent   4  0.703704
4     data dependent   5  0.703704
5     data dependent   6  0.740741
6     data dependent   7  0.666667
7     data dependent